In [2]:
import pandas as pd
import sqlite3

In [3]:
#df = pd.read_csv("output.csv", names=["artist","url","artist_country","location","date_string","insta","spotify","rf_short_description","m_rating","l_rating","comment","emoji_beskrivelse","start_time","new_date_string","start_datetime"])
#df = pd.read_csv("output.csv", names=["artist","url","location","m_rating","start_datetime"])



# Connect to the database
conn = sqlite3.connect('my_database.db')

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv('roskilde_transformed.csv')

# Write the DataFrame to the database
df.to_sql('roskilde_transformed', conn, if_exists='replace', index=False)

263

In [5]:
# Values to put into the different spots
query = '''
    SELECT 
        location || SUBSTR(start_datetime, 1, 13) AS primary_key,
        "+" || SUBSTR(start_datetime, 15, 2) || " " || --get the minute mark each artist start playing
        artist || " " || SUBSTR(m_rating, 1, 1) --get artist and rating given
            AS title
    FROM roskilde_transformed
    WHERE SUBSTR(m_rating, 1, 1) IN ("😌","🤯","👍","🤡")
    '''

# Read the data into a Pandas DataFrame
pd.read_sql(query, conn).head()

,title,primary_key
0,+00 070 Shake 🤡,🚀 Apollo2023-06-29 20
1,+15 Abji_Hypersun Feat. Moraya 😌,Platform2023-06-29 00
2,+15 Adekunle Gold 👍,👩 Eos2023-06-30 15
3,+45 Alice Glass 👍,🚀 Apollo2023-06-30 21
4,+30 Angélique Kidjo 👍,🎪 Arena2023-07-01 19


In [50]:
# ordering of Start- and end times of each day
query = '''
    WITH src AS (
        SELECT DISTINCT
            SUBSTR(start_datetime, 1, 10) AS festival_day,
            SUBSTR(start_datetime, 12, 2) AS hour_playing,
            SUBSTR(start_datetime, 12, 1) = "0" AS is_after_midnight
        FROM roskilde_transformed
    ),

    hours AS (
        SELECT DISTINCT 
            hour_playing AS hours,
            CAST(hour_playing AS int) + 
                CASE WHEN is_after_midnight 
                    THEN 24 
                    ELSE 0 
                END AS ordering
        FROM src
    ),

    before_midnight AS ( 
        SELECT 
            festival_day,
            MIN(hour_playing) AS start_hour,
            MAX(hour_playing) AS end_hour
        FROM src
        WHERE SUBSTR(hour_playing, 1, 1) = "1"
        GROUP BY 1
    ),

    after_midnight AS ( 
        SELECT 
            festival_day,
            MIN(hour_playing) AS start_hour,
            MAX(hour_playing) AS end_hour
        FROM src
        WHERE SUBSTR(hour_playing, 1, 1) = "0"
        GROUP BY 1
    ),

    day_durations AS (
        SELECT * FROM before_midnight
        UNION ALL 
        SELECT * FROM after_midnight
    )

    SELECT festival_day, hours as hour, ordering
    FROM day_durations as dd, hours --cross join
    WHERE hours.hours >= dd.start_hour AND hours.hours <= dd.end_hour --fill out any potential gaps
    ORDER BY 1,3

    '''

# Read the data into a Pandas DataFrame
pd.read_sql(query, conn)[:20]

,festival_day,hour,ordering
0,2023-06-25,10,10
1,2023-06-25,11,11
2,2023-06-25,12,12
3,2023-06-25,13,13
4,2023-06-25,14,14
5,2023-06-25,15,15
6,2023-06-25,16,16
7,2023-06-25,17,17
8,2023-06-25,18,18
9,2023-06-25,19,19


In [ ]:
# get stage x day

# join stage x day x time with the artist values

# for each day
    # pivot stages
    # write to .csv
    # convert to .md
    # append to a README somewhere


